In [135]:
#importing libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import seaborn as sns
import math
import statsmodels.api as sm
import re
import pickle

In [9]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score as f1
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error as rmse,mean_absolute_error as mae, mean_squared_log_error as rmsle,r2_score as r2sc

from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
from sklearn.decomposition import FactorAnalysis

In [10]:
from sklearn.tree import DecisionTreeRegressor as dtr,export_graphviz
from sklearn.model_selection import GridSearchCV as gs,RandomizedSearchCV as rs
from sklearn.metrics import make_scorer
from feature_engine.discretisation import DecisionTreeDiscretiser as dtd
import graphviz

In [11]:
#rmse(squared=True) is rmse otherwise it is mse

## random state

In [12]:
r=84

In [13]:
data = pd.read_csv('googleplaystore.csv')
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [14]:
data.shape

(10841, 13)

In [15]:
target='Rating'         
model= lr(normalize = True)

In [16]:
linreg= lr(normalize=True)

## preprocessing

In [17]:
app_names=data['App']
app_names

0           Photo Editor & Candy Camera & Grid & ScrapBook
1                                      Coloring book moana
2        U Launcher Lite – FREE Live Cool Themes, Hide ...
3                                    Sketch - Draw & Paint
4                    Pixel Draw - Number Art Coloring Book
                               ...                        
10836                                     Sya9a Maroc - FR
10837                     Fr. Mike Schmitz Audio Teachings
10838                               Parkinson Exercices FR
10839                        The SCP Foundation DB fr nn5n
10840        iHoroscope - 2018 Daily Horoscope & Astrology
Name: App, Length: 10841, dtype: object

In [18]:
data["app_no"]=data.index.values.tolist()

In [19]:
data=data[[z for z in data.columns.tolist() if z not in ['App']]].copy()

last updated month column preprocessing

In [20]:
data['Last Updated Month'] = data['Last Updated'].apply(lambda x:re.search('(.*?) ', x).group(1) if x.find(' ') !=-1 else '')

In [21]:
data['Last Updated DOM'] = data['Last Updated'].apply(lambda x:re.search(' (.*?),', x).group(1) if (x.find(' ') !=-1)&(x.find(',') !=-1) else '')

In [22]:
data['Last Updated Year'] = data['Last Updated'].apply(lambda x:re.search(', (.*)', x).group(1) if x.find(', ') !=-1 else '')

In [23]:
data.head()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,0,January,7,2018
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,1,January,15,2018
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,2,August,1,2018
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,3,June,8,2018
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4,June,20,2018


In [24]:
data[data['Last Updated Month']=='']

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year
10472,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN,10472,,,


removing harmful row from dataset

In [25]:
data.drop([10472],inplace=True)

In [26]:
app_names.drop([10472],inplace=True)

In [27]:
data.iloc[10470:10475]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year
10470,COMMUNICATION,3.4,49,4.0M,"10,000+",Free,0,Everyone,Communication,"February 10, 2017",0.1,2.3 and up,10470,February,10,2017
10471,PERSONALIZATION,3.5,1042,404k,"100,000+",Free,0,Everyone,Personalization,"August 5, 2014",3.0.0,4.0.3 and up,10471,August,5,2014
10473,TOOLS,4.2,134203,4.1M,"10,000,000+",Free,0,Everyone,Tools,"August 7, 2018",6.06.14,4.4 and up,10473,August,7,2018
10474,COMMUNICATION,3.4,37,14M,"1,000+",Free,0,Everyone,Communication,"November 21, 2014",2.2.1.5,2.2 and up,10474,November,21,2014
10475,TOOLS,3.9,132,2.6M,"50,000+",Free,0,Everyone,Tools,"May 17, 2017",0.0.9,2.3 and up,10475,May,17,2017


In [28]:
app_names.iloc[10470:10475]

10470                 Jazz Wi-Fi
10471           Xposed Wi-Fi-Pwd
10473    osmino Wi-Fi: free WiFi
10474               Sat-Fi Voice
10475           Wi-Fi Visualizer
Name: App, dtype: object

In [29]:
month_dict={"January": "01", "February": "02", "March": "03",
            "April": "04", "May": "05", "June": "06", "July": "07", "August": "08", 
            "September": "09", "October": "10", "November": "11", "December": "12"}

In [30]:
data.replace({'Last Updated Month':month_dict},inplace=True)

In [31]:
data.head()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,0,01,7,2018
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,1,01,15,2018
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,2,08,1,2018
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,3,06,8,2018
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4,06,20,2018


In [32]:
data[data[target].isnull()==True]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year
23,ART_AND_DESIGN,NaN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up,23,03,7,2018
113,BEAUTY,NaN,182,5.7M,"100,000+",Free,0,Everyone 10+,Beauty,"September 20, 2017",8.0,3.0 and up,113,09,20,2017
123,BEAUTY,NaN,119,3.7M,"50,000+",Free,0,Everyone,Beauty,"July 23, 2018",1.3,4.1 and up,123,07,23,2018
126,BEAUTY,NaN,654,7.4M,"100,000+",Free,0,Teen,Beauty,"July 17, 2018",1.15,4.1 and up,126,07,17,2018
129,BEAUTY,NaN,77,2.9M,"10,000+",Free,0,Mature 17+,Beauty,"August 8, 2017",2.0,2.3 and up,129,08,8,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10824,MEDICAL,NaN,67,82M,"10,000+",Free,0,Everyone,Medical,"July 31, 2018",2.2.2,4.4 and up,10824,07,31,2018
10825,SOCIAL,NaN,7,7.7M,100+,Free,0,Teen,Social,"February 2, 2018",1.0,4.0 and up,10825,02,2,2018
10831,MAPS_AND_NAVIGATION,NaN,38,9.8M,"5,000+",Free,0,Everyone,Maps & Navigation,"June 13, 2018",2.0.148.0,4.0 and up,10831,06,13,2018
10835,BUSINESS,NaN,0,9.6M,10+,Free,0,Everyone,Business,"September 29, 2016",1.1.5,4.0 and up,10835,09,29,2016


## missing values

In [33]:
kval_n=2

In [34]:
def kval(main,step,n=kval_n):

 klist=[]
 if (step==0):
    step=0.0001
 i=main-step*n
 if (main>=0):
  while (i<=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
 else:
  while (i>=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
  

In [35]:
data.isnull().sum()

Category                 0
Rating                1474
Reviews                  0
Size                     0
Installs                 0
Type                     1
Price                    0
Content Rating           0
Genres                   0
Last Updated             0
Current Ver              8
Android Ver              2
app_no                   0
Last Updated Month       0
Last Updated DOM         0
Last Updated Year        0
dtype: int64

missing values in rating will be predicted in test set so we should not worry about those

since null values in above columns are strings they cannot be treated so its better to delete those rows from dataset

In [36]:
delete_index=[]
delete_index.extend(data[data['Type'].isnull()==True].index.values.tolist())
delete_index.extend(data[data['Current Ver'].isnull()==True].index.values.tolist())
delete_index.extend(data[data['Android Ver'].isnull()==True].index.values.tolist())
delete_index

[9148, 15, 1553, 6322, 6803, 7333, 7407, 7730, 10342, 4453, 4490]

In [37]:
data.drop(delete_index,inplace=True)

In [38]:
app_names.drop(delete_index,inplace=True)

In [39]:
app_names.shape,data.shape

((10829,), (10829, 16))

NOW FOR SOME MORE PREPROCESSING

### preprocessing continued

In [40]:
data.head()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,0,01,7,2018
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,1,01,15,2018
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,2,08,1,2018
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,3,06,8,2018
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4,06,20,2018


In [41]:
data['Size_type']=data['Size'].apply(lambda x:x[-1])

In [42]:
data['Size_type'].value_counts()

M    8823
e    1694
k     312
Name: Size_type, dtype: int64

In [43]:
data[data['Size_type']=='e']

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year,Size_type
37,ART_AND_DESIGN,4.1,36639,Varies with device,"5,000,000+",Free,0,Everyone,Art & Design,"July 14, 2018",Varies with device,2.3.3 and up,37,07,14,2018,e
42,ART_AND_DESIGN,4.4,295221,Varies with device,"10,000,000+",Free,0,Everyone,Art & Design,"July 30, 2018",Varies with device,Varies with device,42,07,30,2018,e
52,AUTO_AND_VEHICLES,4.6,17057,Varies with device,"1,000,000+",Free,0,Everyone,Auto & Vehicles,"July 30, 2018",Varies with device,Varies with device,52,07,30,2018,e
67,AUTO_AND_VEHICLES,4.3,40211,Varies with device,"5,000,000+",Free,0,Everyone,Auto & Vehicles,"July 30, 2018",Varies with device,Varies with device,67,07,30,2018,e
68,AUTO_AND_VEHICLES,3.9,356,Varies with device,"100,000+",Free,0,Everyone,Auto & Vehicles,"May 25, 2018",Varies with device,Varies with device,68,05,25,2018,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10713,WEATHER,4.4,3471,Varies with device,"100,000+",Free,0,Everyone,Weather,"July 24, 2018",Varies with device,Varies with device,10713,07,24,2018,e
10725,MAPS_AND_NAVIGATION,3.6,8,Varies with device,"1,000+",Free,0,Everyone,Maps & Navigation,"September 27, 2017",Varies with device,4.4 and up,10725,09,27,2017,e
10765,SOCIAL,3.4,622,Varies with device,"100,000+",Free,0,Mature 17+,Social,"May 23, 2018",Varies with device,Varies with device,10765,05,23,2018,e
10826,SOCIAL,4.0,88486,Varies with device,"5,000,000+",Free,0,Mature 17+,Social,"March 23, 2018",Varies with device,Varies with device,10826,03,23,2018,e


In [44]:
data.drop(['Size'],axis=1,inplace=True)

In [45]:
data.replace({'Size_type':{'e':0,'M':1,'k':2}},inplace=True)

In [46]:
data['Price'].value_counts()

0         10032
$0.99       146
$2.99       129
$1.99        73
$4.99        72
          ...  
$19.90        1
$1.75         1
$14.00        1
$4.85         1
$1.04         1
Name: Price, Length: 92, dtype: int64

In [47]:
data['Price'] = data['Price'].apply(lambda x:x[1:] if x.find('$') !=-1 else x)

In [48]:
data['Type'].value_counts()

Free    10032
Paid      797
Name: Type, dtype: int64

In [49]:
data.replace({'Type':{'Free':0,'Paid':1}},inplace=True)

In [50]:
data.head()

,Category,Rating,Reviews,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year,Size_type
0,ART_AND_DESIGN,4.1,159,"10,000+",0,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,0,01,7,2018,1
1,ART_AND_DESIGN,3.9,967,"500,000+",0,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,1,01,15,2018,1
2,ART_AND_DESIGN,4.7,87510,"5,000,000+",0,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,2,08,1,2018,1
3,ART_AND_DESIGN,4.5,215644,"50,000,000+",0,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,3,06,8,2018,1
4,ART_AND_DESIGN,4.3,967,"100,000+",0,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,4,06,20,2018,1


In [51]:
data['Installs'].value_counts()

1,000,000+        1578
10,000,000+       1252
100,000+          1169
10,000+           1052
1,000+             905
5,000,000+         752
100+               718
500,000+           538
50,000+            478
5,000+             476
100,000,000+       409
10+                385
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
Name: Installs, dtype: int64

In [52]:
data['Installs'] = data['Installs'].apply(lambda x:x[:-1] if x.find('+') !=-1 else x)

In [53]:
for i in data[['Category','Content Rating','Genres','Current Ver','Android Ver']]:
  le = LabelEncoder()
  le.fit(data[i])
  data[i]=le.transform(data[i])

In [54]:
data=data[[z for z in data.columns.tolist() if z not in ['Last Updated']]].copy()

In [55]:
data.head()

,Category,Rating,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year,Size_type
0,0,4.1,159,"10,000",0,0,1,9,118,15,0,01,7,2018,1
1,0,3.9,967,"500,000",0,0,1,12,1018,15,1,01,15,2018,1
2,0,4.7,87510,"5,000,000",0,0,1,9,464,15,2,08,1,2018,1
3,0,4.5,215644,"50,000,000",0,0,4,9,2764,18,3,06,8,2018,1
4,0,4.3,967,"100,000",0,0,1,11,277,20,4,06,20,2018,1


reviews and install has string values,we have to convert them to int

In [56]:
for i in ['Reviews','Installs','Last Updated Month','Last Updated DOM','Last Updated Year']:
    data[i]=data[i].apply(lambda x:x.replace(",",""))
    data[i]=data[i].apply(lambda x:int(x))
    


In [57]:
data['Price']=data['Price'].apply(lambda x:float(x))

In [58]:
data.dtypes

Category                int64
Rating                float64
Reviews                 int64
Installs                int64
Type                    int64
Price                 float64
Content Rating          int64
Genres                  int64
Current Ver             int64
Android Ver             int64
app_no                  int64
Last Updated Month      int64
Last Updated DOM        int64
Last Updated Year       int64
Size_type               int64
dtype: object

#### train and test split

In [59]:
#seperating independent and dependent variables
x = (data[[z for z in data.columns.tolist() if z not in [target]]]).copy()
y = data[target].copy()
x.shape, y.shape

((10829, 14), (10829,))

In [60]:
test_data=data[data['Rating'].isnull()==True].copy()
test_data.head()

,Category,Rating,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year,Size_type
23,0,NaN,61,100000,0,0.0,1,10,118,17,23,3,7,2018,1
113,2,NaN,182,100000,0,0.0,2,14,2587,10,113,9,20,2017,1
123,2,NaN,119,50000,0,0.0,1,14,532,17,123,7,23,2018,1
126,2,NaN,654,100000,0,0.0,4,14,388,17,126,7,17,2018,1
129,2,NaN,77,10000,0,0.0,3,14,1017,8,129,8,8,2017,1


In [61]:
train_data=data[data['Rating'].isnull()==False].copy()
train_data.head()

,Category,Rating,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year,Size_type
0,0,4.1,159,10000,0,0.0,1,9,118,15,0,1,7,2018,1
1,0,3.9,967,500000,0,0.0,1,12,1018,15,1,1,15,2018,1
2,0,4.7,87510,5000000,0,0.0,1,9,464,15,2,8,1,2018,1
3,0,4.5,215644,50000000,0,0.0,4,9,2764,18,3,6,8,2018,1
4,0,4.3,967,100000,0,0.0,1,11,277,20,4,6,20,2018,1


In [62]:
train_x=train_data[x.columns.tolist()].copy()
test_x=test_data[x.columns.tolist()].copy()
train_y=train_data[target].copy()
test_y=test_data[target].copy()

In [63]:
train_x,val_x,train_y,val_y= train_test_split(train_x, train_y, test_size=1/4,random_state=r).copy()

In [64]:
model.fit(train_x,train_y)
pred=model.predict(val_x)
rmse(val_y,pred,squared=True)

0.2566995338500597

## feature binning

In [65]:
data.dtypes

Category                int64
Rating                float64
Reviews                 int64
Installs                int64
Type                    int64
Price                 float64
Content Rating          int64
Genres                  int64
Current Ver             int64
Android Ver             int64
app_no                  int64
Last Updated Month      int64
Last Updated DOM        int64
Last Updated Year       int64
Size_type               int64
dtype: object

In [66]:
data.head()

,Category,Rating,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated Month,Last Updated DOM,Last Updated Year,Size_type
0,0,4.1,159,10000,0,0.0,1,9,118,15,0,1,7,2018,1
1,0,3.9,967,500000,0,0.0,1,12,1018,15,1,1,15,2018,1
2,0,4.7,87510,5000000,0,0.0,1,9,464,15,2,8,1,2018,1
3,0,4.5,215644,50000000,0,0.0,4,9,2764,18,3,6,8,2018,1
4,0,4.3,967,100000,0,0.0,1,11,277,20,4,6,20,2018,1


In [67]:
contlist=[]
for i in x:
 if ((type(x[i])==float)|(len(x[i].value_counts())>6)):
    contlist.append(i)

In [68]:
contlist

['Category',
 'Reviews',
 'Installs',
 'Price',
 'Genres',
 'Current Ver',
 'Android Ver',
 'app_no',
 'Last Updated Month',
 'Last Updated DOM',
 'Last Updated Year']

In [69]:
catlist=[t for t in x.columns.tolist() if t not in contlist]
catlist

['Type', 'Content Rating', 'Size_type']

decision tree regressor for binning contlist columns

In [70]:
dt=dtr(random_state=r)

In [71]:
for k in contlist:
 val_x1=val_x[k].to_frame().copy()
 train_x1=train_x[k].to_frame().copy() 

 
 dtdmodel=dtd(variables=k,
    regression = True,
    random_state=r)
 dtdmodel.fit(train_x1,train_y)
 x[str(k)+"-Tree"]=dtdmodel.transform(pd.DataFrame(x[k])).copy()

In [72]:
x.head()

,Category,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,...,Reviews-Tree,Installs-Tree,Price-Tree,Genres-Tree,Current Ver-Tree,Android Ver-Tree,app_no-Tree,Last Updated Month-Tree,Last Updated DOM-Tree,Last Updated Year-Tree
0,0,159,10000,0,0.0,1,9,118,15,0,...,4.037250,4.046013,4.18732,4.281014,4.177877,4.246893,4.371429,4.149837,4.203851,4.244192
1,0,967,500000,0,0.0,1,12,1018,15,1,...,4.037250,4.203095,4.18732,4.281014,4.177877,4.246893,4.371429,4.149837,4.142905,4.244192
2,0,87510,5000000,0,0.0,1,9,464,15,2,...,4.286289,4.246211,4.18732,4.281014,4.177877,4.246893,4.371429,4.287975,4.228280,4.244192
3,0,215644,50000000,0,0.0,4,9,2764,18,3,...,4.356677,4.340271,4.18732,4.281014,4.269748,4.246893,4.371429,4.193206,4.142905,4.244192
4,0,967,100000,0,0.0,1,11,277,20,4,...,4.037250,4.101752,4.18732,4.281014,4.177877,4.149786,4.371429,4.193206,4.142905,4.244192


In [73]:
x.drop(['app_no-Tree'],axis=1,inplace=True)

In [74]:
train_x=x[data['Rating'].isnull()==False].copy()
test_x=x[data['Rating'].isnull()==True].copy()

In [75]:
train_x.shape,train_y.shape

((9360, 24), (7020,))

In [76]:
train_x,val_x= train_test_split(train_x ,test_size=1/4,random_state=r).copy()

# backward feature

In [77]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.metrics import make_scorer
def backfeat(nfeat):
 sfs1 = sfs(model, k_features=nfeat, forward=False)
 sfs0 = sfs1.fit(train_x, train_y)
 featlist = list(sfs0.k_feature_names_)
 featlist
 
 return featlist

In [78]:
 #LARGE EXECUTION TIME ESPECIALLY TO REDUCE TO SMALLER NO OF COLUMNS
 topfeatlist=[]
 topfeatscorelist=[]
 score=0
 min_=1000000000
 for j in range(4,len(x.columns.tolist())):
  x1=x[backfeat(j)].copy()
  trainx=x[data['Rating'].isnull()==False].copy()
  testx=x[data['Rating'].isnull()==True].copy()
  trainy=y[data['Rating'].isnull()==False].copy()
  testy=y[data['Rating'].isnull()==True].copy()
  trainx,valx,trainy,valy = train_test_split(trainx,trainy,test_size=1/4,random_state= r).copy()

  model.fit(trainx, trainy)
  pred=model.predict(valx)
  score=rmse(valy,pred,squared=True)
  if(score<min_):
    min_=score
    topn_feat=j
 topfeatlist.append(topn_feat)
 topfeatscorelist.append(min_) 

In [79]:
topfeatlist=[]
topfeatscorelist=[]
step=math.ceil(math.sqrt(len(x.columns)/2))
score=0
min_=1000000000
if (topn_feat+step)>len(x.columns):
 upperlimit=len(x.columns)
else:
 upperlimit=(topn_feat+step)
for i in range((topn_feat-step+1),upperlimit):
  x1=x[backfeat(j)].copy()
  trainx=x[data['Rating'].isnull()==False].copy()
  testx=x[data['Rating'].isnull()==True].copy()
  trainy=y[data['Rating'].isnull()==False].copy()
  testy=y[data['Rating'].isnull()==True].copy()
  trainx,valx,trainy,valy = train_test_split(trainx,trainy,test_size=1/4,random_state= r).copy()


  model.fit(trainx, trainy)
  pred=model.predict(valx)
  score=rmse(valy,pred,squared=True)
  if(score<min_):
    min_=score
    topn_feat=j
topfeatlist.append(topn_feat)
topfeatscorelist.append(min_)  

In [80]:
topfeatlist,topfeatscorelist

([23], [0.23594296846525725])

In [81]:
backfeatcols=backfeat(topfeatlist[0])
backfeatcols

['Category',
 'Reviews',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Current Ver',
 'Android Ver',
 'app_no',
 'Last Updated DOM',
 'Last Updated Year',
 'Size_type',
 'Category-Tree',
 'Reviews-Tree',
 'Installs-Tree',
 'Price-Tree',
 'Genres-Tree',
 'Current Ver-Tree',
 'Android Ver-Tree',
 'Last Updated Month-Tree',
 'Last Updated DOM-Tree',
 'Last Updated Year-Tree']

In [82]:
  x=x[backfeatcols].copy()
  train_x=x[data['Rating'].isnull()==False].copy()
  test_x=x[data['Rating'].isnull()==True].copy()
  train_y=y[data['Rating'].isnull()==False].copy()
  test_y=y[data['Rating'].isnull()==True].copy()
  train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=1/4,random_state= r).copy()

In [83]:
original_col=['Category',
 'Reviews',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Current Ver',
 'Android Ver',
 'app_no',
 'Last Updated DOM',
 'Last Updated Year',
 'Size_type']


In [84]:
  model.fit(trainx[original_col], trainy)
  pred=model.predict(valx[original_col])
  score=rmse(valy,pred,squared=True)
  score

0.2564323466023756

In [85]:
exclude=['Current Ver-Tree',
 'Android Ver-Tree',
 'Last Updated Month-Tree',
 'Last Updated DOM-Tree',
 'Last Updated Year-Tree']
consider=[t for t in backfeatcols if t not in exclude]
consider

['Category',
 'Reviews',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Current Ver',
 'Android Ver',
 'app_no',
 'Last Updated DOM',
 'Last Updated Year',
 'Size_type',
 'Category-Tree',
 'Reviews-Tree',
 'Installs-Tree',
 'Price-Tree',
 'Genres-Tree']

In [86]:
  model.fit(trainx[consider], trainy)
  pred=model.predict(valx[consider])
  score=rmse(valy,pred,squared=True)
  score

0.23601943634745748

consider not much more than least rmse score attained after backfeatcols which means that estimation that exclude columns are not extremely important in our analysis but does increase computation time hence it should be excluded

In [87]:
  x=x[consider].copy()
  train_x=x[data['Rating'].isnull()==False].copy()
  test_x=x[data['Rating'].isnull()==True].copy()
  train_y=y[data['Rating'].isnull()==False].copy()
  test_y=y[data['Rating'].isnull()==True].copy()
  train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=1/4,random_state= r).copy()

# xgboost

In [88]:
def kvall(main,step,n=kval_n):

 klist=[]
 if (step<0.0001):
    step=0.0001
 if (step==0):
    step=0.0001
 i=main-step*n
 if (main>=0):
  while (i<=main):
    
    klist.append(i)
    i=i+step
  return klist
 else:
  while (i>main+step*n):
    
    klist.append(i)
    i=i+step
  return klist

In [89]:
def kvalg(main,step,n=kval_n):

 klist=[]

 if (step==0):
    step=0.0001
 i=main
 if (main>=0):
  while (i<=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist
 else:
  while (i>=main+step*n):
    
    klist.append(i)
    i=i+step
  return klist

In [90]:
depth=kval(100,10,10)
decimal=kval(0.25,(0.25/5),4)
subdecimal=kval(0.05,(0.05/5),4)

In [91]:
import xgboost as xgb
from xgboost import XGBRegressor as xgbr

In [92]:
#xgbmdecstep steps the values of hyperparameter (which accepts float values) in kval
xgbmdecstep=3
xgbmdecbreaker=2*(xgbmdecstep+1)

xgbmgsdecstep=int(2)
xgbmgsdecbreaker=1*(xgbmgsdecstep+1)

earlystopping=1

In [93]:
xgbm=xgbr(random_state=r,verbosity=0)

In [94]:
default={'learning_rate':0.03,'subsample':1,'colsample_bylevel':0.9,'colsample_bynode':0.9,'colsample_bytree':0.9}

try both decimal and subdecimal

In [95]:
xgbmdict={'max_depth':kval(100,5,10),
'n_estimators':kval(100,5,10),
 'min_samples_split':decimal,
 'min_weight_fraction_leaf':decimal,
 'min_impurity_decrease':decimal,
 'min_impurity_split':decimal,
'loss':['ls', 'lad', 'huber', 'quantile'],
'learning_rate':kval(0.1,0.1/10,5),
'n_estimators':kval(100,10,5),
'subsample':kval(0.9,0.02,5),
'criterion':['friedman_mse', 'mse'],
'colsample_bylevel':kval(0.75,0.05,4),
'colsample_bynode':kval(0.75,0.05,4),
'colsample_bytree':kval(0.75,0.05,4),
'gamma':kval(0.2,0.05,5),
'reg_alpha':kval(0.2,0.05,5),
'reg_lambda':kval(0.2,0.05,5)}

In [96]:
xgbmbest=xgbmdict.copy()
xgbmmodel = rs(xgbm, xgbmdict,random_state=r)
xgbmmodel.fit(train_x,train_y)
xgbmbest=xgbmmodel.best_estimator_.get_params().copy()

In [97]:
xgbmbest1=xgbmbest.copy()
xgbmbest1['eval_metric']="rmse"
xgbmbest1

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.55,
 'colsample_bynode': 0.6000000000000001,
 'colsample_bytree': 0.6500000000000001,
 'gamma': 0.39999999999999997,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.05,
 'max_delta_step': 0,
 'max_depth': 75,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 130,
 'n_jobs': 8,
 'num_parallel_tree': 1,
 'random_state': 84,
 'reg_alpha': 0.10000000000000002,
 'reg_lambda': 0.15000000000000002,
 'scale_pos_weight': 1,
 'subsample': 0.8600000000000001,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': 0,
 'min_weight_fraction_leaf': 0.39999999999999997,
 'min_samples_split': 0.2,
 'min_impurity_split': 0.09999999999999999,
 'min_impurity_decrease': 0.44999999999999996,
 'loss': 'ls',
 'criterion': 'friedman_mse',
 'eval_metric': 'rmse'}

In [98]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgbmodel = xgb.train(dtrain=dtrain,num_boost_round=10000,evals=watchlist, early_stopping_rounds=earlystopping,
      maximize=False, verbose_eval=10 ,params=xgbmbest1)
xbgmpredict={'ntree_limit':xgbmodel.best_ntree_limit}
print('Modeling RMSE %.5f' % xgbmodel.best_score)

[0]	train-rmse:3.54469	val-rmse:3.54432
[10]	train-rmse:2.15577	val-rmse:2.15975
[20]	train-rmse:1.33360	val-rmse:1.35117
[30]	train-rmse:0.85157	val-rmse:0.89622
[40]	train-rmse:0.57110	val-rmse:0.65759
[50]	train-rmse:0.41699	val-rmse:0.54239
[60]	train-rmse:0.32787	val-rmse:0.49404
[70]	train-rmse:0.27948	val-rmse:0.47421
[80]	train-rmse:0.25277	val-rmse:0.46613
[90]	train-rmse:0.23577	val-rmse:0.46187
[100]	train-rmse:0.22421	val-rmse:0.45993
[107]	train-rmse:0.21903	val-rmse:0.45949
Modeling RMSE 0.45945


In [99]:
xgbmmodel = xgbr(**xgbmbest)
xgbmmodel.fit(train_x,train_y)
pred=xgbmmodel.predict(val_x,**xbgmpredict)
predtr=xgbmmodel.predict(train_x,**xbgmpredict)
[rmse(val_y, pred,squared=True),rmse(train_y, predtr,squared=True)]

[0.21109459776950396, 0.048282797954806334]

In [100]:
xgbmbestgs=xgbmbest.copy()
for j in xgbmbestgs:
    xgbmbestgs[j]=[xgbmbestgs[j]]
for j in [x for x in xgbmbestgs if x not in xgbmdict]:
    xgbmbestgs.pop(j)

In [101]:
xgbmdict={'loss':['ls', 'lad', 'huber', 'quantile'],
         'criterion':['friedman_mse', 'mse'],
 'n_estimators':kval(xgbmbest['n_estimators'],5,[5 if xgbmbest['n_estimators']>5 else x for x in [xgbmbest['n_estimators']]][0]),
 'max_depth':kval(xgbmbest['max_depth'],5,[5 if xgbmbest['max_depth']>5 else x for x in [xgbmbest['max_depth']]][0]),
 'min_samples_split':[xgbmbest['min_samples_split']],
 'min_weight_fraction_leaf':kval(xgbmbest['min_weight_fraction_leaf'],xgbmbest['min_weight_fraction_leaf']/xgbmgsdecbreaker,xgbmgsdecstep),
 'min_impurity_decrease':[xgbmbest['min_weight_fraction_leaf']],
 'min_impurity_split':[xgbmbest['min_impurity_split']],
  'subsample':[xgbmbest['subsample']],
  'learning_rate':[xgbmbest['learning_rate']],
  'colsample_bylevel':[xgbmbest['colsample_bylevel']] ,
'colsample_bytree':[xgbmbest['colsample_bytree']],
'colsample_bynode':[xgbmbest['colsample_bynode']],
'gamma':kval(xgbmbest['gamma'],xgbmbest['gamma']/xgbmgsdecbreaker,xgbmgsdecstep),
'reg_alpha':[xgbmbest['reg_alpha']],
'reg_lambda':[xgbmbest['reg_lambda']],
'objective':[xgbmbest['objective']]}

In [102]:
xgmbest=xgbmdict.copy()
for j in ['loss','criterion','n_estimators','max_depth','gamma','min_weight_fraction_leaf'] :
 tree_para ={**xgbmbestgs,j:xgbmdict[j]}
 xgbmmodel = gs(xgbm, tree_para)
 xgbmmodel.fit(train_x,train_y)
 xgbmbest[j]=xgbmmodel.best_estimator_.get_params()[j]

In [103]:
xgbmbest1=xgbmbest.copy()
xgbmbest1['eval_metric']="rmse"
xgbmbest1=xgbmbest.copy()
for j in xgbmbest1:
 xgbmbest1[j]=[xgbmbest[j][0] if type(xgbmbest[j])==list else x for x in [xgbmbest[j]]][0]
xgbmbest1

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.55,
 'colsample_bynode': 0.6000000000000001,
 'colsample_bytree': 0.6500000000000001,
 'gamma': 0.1333333333333333,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.05,
 'max_delta_step': 0,
 'max_depth': 50,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 155,
 'n_jobs': 8,
 'num_parallel_tree': 1,
 'random_state': 84,
 'reg_alpha': 0.10000000000000002,
 'reg_lambda': 0.15000000000000002,
 'scale_pos_weight': 1,
 'subsample': 0.8600000000000001,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': 0,
 'min_weight_fraction_leaf': 0.1333333333333333,
 'min_samples_split': 0.2,
 'min_impurity_split': 0.09999999999999999,
 'min_impurity_decrease': 0.44999999999999996,
 'loss': 'ls',
 'criterion': 'friedman_mse'}

In [104]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgbmodel = xgb.train(dtrain=dtrain,num_boost_round=10000,evals=watchlist, early_stopping_rounds=earlystopping,
      maximize=False, verbose_eval=10 ,params=xgbmbest1)
xbgmpredict={'ntree_limit':xgbmodel.best_ntree_limit}
print('Modeling RMSE %.5f' % xgbmodel.best_score)

[0]	train-rmse:3.54469	val-rmse:3.54437
[10]	train-rmse:2.15578	val-rmse:2.15997
[20]	train-rmse:1.33337	val-rmse:1.35126
[30]	train-rmse:0.84881	val-rmse:0.89780
[40]	train-rmse:0.56378	val-rmse:0.65669
[50]	train-rmse:0.40421	val-rmse:0.54124
[60]	train-rmse:0.30524	val-rmse:0.49024
[70]	train-rmse:0.24727	val-rmse:0.46915
[80]	train-rmse:0.21082	val-rmse:0.46054
[90]	train-rmse:0.18630	val-rmse:0.45707
[97]	train-rmse:0.17546	val-rmse:0.45557
Modeling RMSE 0.45557


In [105]:
xgbmmodel = xgbr(**xgbmbest1)
xgbmmodel.fit(train_x,train_y)
pred=xgbmmodel.predict(val_x,**xbgmpredict)
predtr=xgbmmodel.predict(train_x,**xbgmpredict)
[rmse(val_y, pred,squared=True),rmse(train_y, predtr,squared=True)]

[0.2075473392131459, 0.030786490992756783]

In [106]:
xgbmbest={'criterion':[xgbmbest['criterion']],
 'loss':[xgbmbest['loss']],
  'max_depth':[xgbmbest['max_depth']],
 'n_estimators':[xgbmbest['n_estimators']],
 'min_weight_fraction_leaf':[xgbmbest['min_weight_fraction_leaf']],
 'min_impurity_decrease':kval(xgbmbest['min_impurity_decrease'],xgbmbest['min_impurity_decrease']/xgbmdecbreaker,xgbmdecstep),
 'min_impurity_split':kval(xgbmbest['min_impurity_split'],xgbmbest['min_impurity_split']/xgbmdecbreaker,xgbmdecstep),
  'min_samples_split':kval(xgbmbest['min_samples_split'],xgbmbest['min_samples_split']/xgbmdecbreaker,xgbmdecstep),
  'subsample':[x for x in kval(xgbmbest['subsample'],abs(xgbmbest['subsample']-default['subsample'])/xgbmdecbreaker,xgbmdecstep) if x<1],
 'learning_rate':[xgbmbest['learning_rate']],
'colsample_bylevel':[x for x in kval(xgbmbest['colsample_bylevel'],abs(xgbmbest[ 'colsample_bylevel']-default[ 'colsample_bylevel'])/xgbmdecbreaker,xgbmdecstep) if x<1] ,
'colsample_bytree':[x for x in kval(xgbmbest['colsample_bytree'],abs(xgbmbest['colsample_bytree']-default['colsample_bytree'])/xgbmdecbreaker,xgbmdecstep) if x<1],
'colsample_bynode':[x for x in kval(xgbmbest['colsample_bynode'],abs(xgbmbest[ 'colsample_bynode']-default['colsample_bynode'])/xgbmdecbreaker,xgbmdecstep) if x<1],
'gamma':[xgbmbest['gamma']],
'reg_alpha':kval(xgbmbest['reg_alpha'],xgbmbest['reg_alpha']/xgbmdecbreaker,xgbmdecstep),
'reg_lambda':kval(xgbmbest['reg_lambda'],xgbmbest['reg_lambda']/xgbmdecbreaker,xgbmdecstep)}

In [107]:
for j in [x for x in [t for t in xgbmbest if type(xgbmbest[t][0]) !=str] if xgbmbest[x][0]<0]:
  xgbmbest[j]=[x for x in xgbmbest[j] if x>0] 
xgbmmodel = rs(xgbm, xgbmbest,random_state=r)
xgbmmodel.fit(train_x,train_y)
xgbmbest=xgbmmodel.best_estimator_.get_params().copy()

In [108]:
xgbmbest

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.4625000000000001,
 'colsample_bynode': 0.6000000000000001,
 'colsample_bytree': 0.6500000000000001,
 'gamma': 0.1333333333333333,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.05,
 'max_delta_step': 0,
 'max_depth': 50,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 155,
 'n_jobs': 8,
 'num_parallel_tree': 1,
 'random_state': 84,
 'reg_alpha': 0.08750000000000001,
 'reg_lambda': 0.20625,
 'scale_pos_weight': 1,
 'subsample': 0.9124999999999999,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': 0,
 'min_weight_fraction_leaf': 0.1333333333333333,
 'min_samples_split': 0.175,
 'min_impurity_split': 0.06249999999999999,
 'min_impurity_decrease': 0.28125,
 'loss': 'ls',
 'criterion': 'friedman_mse'}

In [109]:
xgbmbest1=xgbmbest.copy()
xgbmbest1['eval_metric']="rmse"
xgbmbest1=xgbmbest.copy()
for j in xgbmbest1:
 xgbmbest1[j]=[xgbmbest[j][0] if type(xgbmbest[j])==list else x for x in [xgbmbest[j]]][0]
xgbmbest1

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.4625000000000001,
 'colsample_bynode': 0.6000000000000001,
 'colsample_bytree': 0.6500000000000001,
 'gamma': 0.1333333333333333,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.05,
 'max_delta_step': 0,
 'max_depth': 50,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 155,
 'n_jobs': 8,
 'num_parallel_tree': 1,
 'random_state': 84,
 'reg_alpha': 0.08750000000000001,
 'reg_lambda': 0.20625,
 'scale_pos_weight': 1,
 'subsample': 0.9124999999999999,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': 0,
 'min_weight_fraction_leaf': 0.1333333333333333,
 'min_samples_split': 0.175,
 'min_impurity_split': 0.06249999999999999,
 'min_impurity_decrease': 0.28125,
 'loss': 'ls',
 'criterion': 'friedman_mse'}

In [110]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgbmodel = xgb.train(dtrain=dtrain,num_boost_round=10000,evals=watchlist, early_stopping_rounds=earlystopping,
      maximize=False, verbose_eval=10 ,params=xgbmbest1)
xbgmpredict={'ntree_limit':xgbmodel.best_ntree_limit}
print('Modeling RMSE %.5f' % xgbmodel.best_score)

[0]	train-rmse:3.54481	val-rmse:3.54434
[10]	train-rmse:2.15733	val-rmse:2.15977
[20]	train-rmse:1.33819	val-rmse:1.35231
[30]	train-rmse:0.85598	val-rmse:0.89783
[40]	train-rmse:0.56944	val-rmse:0.65856
[50]	train-rmse:0.40397	val-rmse:0.54270
[60]	train-rmse:0.29764	val-rmse:0.49322
[70]	train-rmse:0.23689	val-rmse:0.47248
[80]	train-rmse:0.20074	val-rmse:0.46371
[90]	train-rmse:0.17841	val-rmse:0.45968
[100]	train-rmse:0.16275	val-rmse:0.45727
[107]	train-rmse:0.15585	val-rmse:0.45662
Modeling RMSE 0.45659


In [111]:
xgbmmodel = xgbr(**xgbmbest1)
xgbmmodel.fit(train_x,train_y)
pred=xgbmmodel.predict(val_x,**xbgmpredict)
predtr=xgbmmodel.predict(train_x,**xbgmpredict)

[rmse(val_y, pred,squared=True),rmse(train_y, predtr,squared=True)]

[0.2084730228130948, 0.024624839858283173]

In [112]:
for j in xgbmbest:
 xgbmbest[j]=[xgbmbest[j]]
xgbmbestgs=xgbmbest.copy()
for j in [x for x in xgbmbestgs if x not in xgbmdict]:
 print(xgbmbestgs.pop(j))

[0.5]
['gbtree']
[-1]
['gain']
['']
[0]
[1]
[nan]
['()']
[8]
[1]
[84]
[1]
['exact']
[1]
[0]


In [113]:
xgbmbest={'loss':xgbmbest['loss'],
         'criterion':xgbmbest['criterion'],
 'n_estimators':kvalg(xgbmbest['n_estimators'][0],2,[5 if xgbmbest['n_estimators'][0]>5 else x for x in [xgbmbest['n_estimators']][0]][0]),
 'max_depth':kvall(xgbmbest['max_depth'][0],2,[5 if xgbmbest['max_depth'][0]>5 else x for x in [xgbmbest['max_depth']][0]][0]),
 'min_samples_split':xgbmbest['min_samples_split'],
 'min_weight_fraction_leaf':kvalg(xgbmbest['min_weight_fraction_leaf'][0],xgbmbest['min_weight_fraction_leaf'][0]/xgbmgsdecbreaker,xgbmgsdecstep),
 'min_impurity_decrease':xgbmbest['min_weight_fraction_leaf'],
 'min_impurity_split':xgbmbest['min_impurity_split'],
  'subsample':xgbmbest['subsample'],
  'learning_rate':xgbmbest['learning_rate'],
  'colsample_bylevel':xgbmbest['colsample_bylevel'],
'colsample_bytree':xgbmbest['colsample_bytree'],
'colsample_bynode':xgbmbest['colsample_bynode'],
'gamma':kvalg(xgbmbest['gamma'][0],xgbmbest['gamma'][0]/xgbmgsdecbreaker,xgbmgsdecstep),
'reg_alpha':xgbmbest['reg_alpha'],
'reg_lambda':xgbmbest['reg_lambda'],
'objective':xgbmbest['objective']}

In [114]:
for j in ['n_estimators','max_depth','gamma','min_weight_fraction_leaf'] :
 tree_para ={**xgbmbestgs,j:xgbmbest[j]}
 xgbmmodel = gs(xgbm, tree_para)
 xgbmmodel.fit(train_x,train_y)
 xgbmbest[j]=[xgbmmodel.best_estimator_.get_params()[j]]

In [115]:
xgbmbest1=xgbmbest.copy()
xgbmbest1['eval_metric']="rmse"
xgbmbest1=xgbmbest.copy()
for j in xgbmbest1:
 xgbmbest1[j]=[xgbmbest[j][0] if type(xgbmbest[j])==list else x for x in [xgbmbest[j]]][0]
xgbmbest1

{'loss': 'ls',
 'criterion': 'friedman_mse',
 'n_estimators': 165,
 'max_depth': 48,
 'min_samples_split': 0.175,
 'min_weight_fraction_leaf': 0.1333333333333333,
 'min_impurity_decrease': 0.1333333333333333,
 'min_impurity_split': 0.06249999999999999,
 'subsample': 0.9124999999999999,
 'learning_rate': 0.05,
 'colsample_bylevel': 0.4625000000000001,
 'colsample_bytree': 0.6500000000000001,
 'colsample_bynode': 0.6000000000000001,
 'gamma': 0.1333333333333333,
 'reg_alpha': 0.08750000000000001,
 'reg_lambda': 0.20625,
 'objective': 'reg:squarederror'}

In [116]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgbmodel = xgb.train(dtrain=dtrain,num_boost_round=10000,evals=watchlist, early_stopping_rounds=earlystopping,
      maximize=False, verbose_eval=10 ,params=xgbmbest1)
xbgmpredict={'ntree_limit':xgbmodel.best_ntree_limit}
print('Modeling RMSE %.5f' % xgbmodel.best_score)

[0]	train-rmse:3.54492	val-rmse:3.54440
[10]	train-rmse:2.15633	val-rmse:2.16158
[20]	train-rmse:1.33435	val-rmse:1.35308
[30]	train-rmse:0.85284	val-rmse:0.89882
[40]	train-rmse:0.56634	val-rmse:0.65793
[50]	train-rmse:0.40123	val-rmse:0.54307
[60]	train-rmse:0.30229	val-rmse:0.49228
[70]	train-rmse:0.24521	val-rmse:0.47097
[80]	train-rmse:0.20231	val-rmse:0.46185
[90]	train-rmse:0.17893	val-rmse:0.45844
[94]	train-rmse:0.17322	val-rmse:0.45729
Modeling RMSE 0.45729


In [117]:
xgbmmodel = xgbr(**xgbmbest1)
xgbmmodel.fit(train_x,train_y)
pred=xgbmmodel.predict(val_x,**xbgmpredict)
predtr=xgbmmodel.predict(train_x,**xbgmpredict)
[rmse(val_y, pred,squared=True),rmse(train_y, predtr,squared=True)]

[0.20911586830233048, 0.030004648575588152]

In [118]:
xgbmbest={'criterion':xgbmbest['criterion'],
 'loss':xgbmbest['loss'],
  'max_depth':xgbmbest['max_depth'],
 'n_estimators':xgbmbest['n_estimators'],
 'min_weight_fraction_leaf':xgbmbest['min_weight_fraction_leaf'],
 'min_impurity_decrease':kval(xgbmbest['min_impurity_decrease'][0],xgbmbest['min_impurity_decrease'][0]/xgbmdecbreaker,xgbmdecstep),
 'min_impurity_split':kval(xgbmbest['min_impurity_split'][0],xgbmbest['min_impurity_split'][0]/xgbmdecbreaker,xgbmdecstep),
  'min_samples_split':kval(xgbmbest['min_samples_split'][0],xgbmbest['min_samples_split'][0]/xgbmdecbreaker,xgbmdecstep),
  'subsample':[x for x in kval(xgbmbest['subsample'][0],xgbmbest[ 'subsample'][0]/xgbmdecbreaker,xgbmdecstep) if x<1],
 'learning_rate':kval(xgbmbest['learning_rate'][0],xgbmbest['learning_rate'][0]/xgbmdecbreaker,xgbmdecstep),
'colsample_bylevel':[x for x in kval(xgbmbest['colsample_bylevel'][0],xgbmbest[ 'colsample_bylevel'][0]/xgbmdecbreaker,xgbmdecstep) if x<1] ,
'colsample_bytree':[x for x in kval(xgbmbest['colsample_bytree'][0],xgbmbest['colsample_bytree'][0]/xgbmdecbreaker,xgbmdecstep) if x<1],
'colsample_bynode':[x for x in kval(xgbmbest['colsample_bynode'][0],xgbmbest['colsample_bynode'][0]/xgbmdecbreaker,xgbmdecstep) if x<1],
'gamma':xgbmbest['gamma'],
'reg_alpha':kval(xgbmbest['reg_alpha'][0],xgbmbest['reg_alpha'][0]/xgbmdecbreaker,xgbmdecstep),
'reg_lambda':kval(xgbmbest['reg_lambda'][0],xgbmbest['reg_lambda'][0]/xgbmdecbreaker,xgbmdecstep)}



In [119]:
for j in [x for x in [t for t in xgbmbest if type(xgbmbest[t][0]) !=str] if xgbmbest[x][0]<0]:
  xgbmbest[j]=[x for x in xgbmbest[j] if x>0]  

In [120]:
xgbmmodel = rs(xgbm, xgbmbest,random_state=r)
xgbmmodel.fit(train_x,train_y)
xgbmbest=xgbmmodel.best_estimator_.get_params().copy() 

In [121]:
xgbmbest1=xgbmbest.copy()
xgbmbest1['eval_metric']="rmse"
xgbmbest1=xgbmbest.copy()
for j in xgbmbest1:
 xgbmbest1[j]=[xgbmbest[j][0] if type(xgbmbest[j])==list else x for x in [xgbmbest[j]]][0]
xgbmbest1

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.5203125000000001,
 'colsample_bynode': 0.75,
 'colsample_bytree': 0.48750000000000016,
 'gamma': 0.1333333333333333,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.049999999999999996,
 'max_delta_step': 0,
 'max_depth': 48,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 165,
 'n_jobs': 8,
 'num_parallel_tree': 1,
 'random_state': 84,
 'reg_alpha': 0.08750000000000001,
 'reg_lambda': 0.1546875,
 'scale_pos_weight': 1,
 'subsample': 0.5703124999999999,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': 0,
 'min_weight_fraction_leaf': 0.1333333333333333,
 'min_samples_split': 0.13125,
 'min_impurity_split': 0.0625,
 'min_impurity_decrease': 0.11666666666666664,
 'loss': 'ls',
 'criterion': 'friedman_mse'}

In [122]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
watchlist = [(dtrain, 'train'), (dval, 'val')]
xgbmodel = xgb.train(dtrain=dtrain,num_boost_round=10000,evals=watchlist, early_stopping_rounds=earlystopping,
      maximize=False, verbose_eval=10 ,params=xgbmbest1)
xbgmpredict={'ntree_limit':xgbmodel.best_ntree_limit}
print('Modeling RMSE %.5f' % xgbmodel.best_score)

[0]	train-rmse:3.54503	val-rmse:3.54449
[10]	train-rmse:2.15824	val-rmse:2.15923
[20]	train-rmse:1.34129	val-rmse:1.35311
[30]	train-rmse:0.86851	val-rmse:0.89927
[40]	train-rmse:0.60083	val-rmse:0.66025
[50]	train-rmse:0.44839	val-rmse:0.54496
[60]	train-rmse:0.35562	val-rmse:0.49438
[70]	train-rmse:0.29425	val-rmse:0.47279
[80]	train-rmse:0.25722	val-rmse:0.46276
[90]	train-rmse:0.23106	val-rmse:0.45836
[91]	train-rmse:0.22815	val-rmse:0.45800
Modeling RMSE 0.45800


In [123]:
xgbmmodel = xgbr(**xgbmbest)
xgbmmodel.fit(train_x,train_y)
pred=xgbmmodel.predict(val_x,**xbgmpredict)
predtr=xgbmmodel.predict(train_x,**xbgmpredict)
[rmse(val_y, pred,squared=True),rmse(train_y, predtr,squared=True)]

[0.20976125813202184, 0.052050869504895306]

In [124]:
for j in xgbmbest:
    xgbmbest[j]=[xgbmbest[j]]

In [125]:
#feature importance against each variable
pd.Series(xgbmmodel.feature_importances_, index=x.columns)

Category             0.059180
Reviews              0.079301
Installs             0.058117
Type                 0.036927
Price                0.049717
Content Rating       0.041754
Genres               0.049524
Current Ver          0.062270
Android Ver          0.051897
app_no               0.068508
Last Updated DOM     0.059948
Last Updated Year    0.058614
Size_type            0.037917
Category-Tree        0.047077
Reviews-Tree         0.067919
Installs-Tree        0.069753
Price-Tree           0.054474
Genres-Tree          0.047104
dtype: float32

In [126]:
for j in xgbmbest:
    xgbmbest[j]=xgbmbest[j][0]

## TEST

In [127]:
xgbmmodel = xgbr(**xgbmbest)
xgbmmodel.fit(train_x,train_y)
pred=xgbmmodel.predict(test_x,**xbgmpredict)

predictions

In [128]:
pred[:5]

array([4.2113185, 4.0272827, 4.3678255, 4.1624713, 4.1487966],
      dtype=float32)

In [129]:
test_x.head()

,Category,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated DOM,Last Updated Year,Size_type,Category-Tree,Reviews-Tree,Installs-Tree,Price-Tree,Genres-Tree
23,0,61,100000,0,0.0,1,10,118,17,23,7,2018,1,4.397561,4.03725,4.101752,4.18732,4.281014
113,2,182,100000,0,0.0,2,14,2587,10,113,20,2017,1,4.247126,4.03725,4.101752,4.18732,4.281014
123,2,119,50000,0,0.0,1,14,532,17,123,23,2018,1,4.247126,4.03725,4.046013,4.18732,4.281014
126,2,654,100000,0,0.0,4,14,388,17,126,17,2018,1,4.247126,4.03725,4.101752,4.18732,4.281014
129,2,77,10000,0,0.0,3,14,1017,8,129,8,2017,1,4.247126,4.03725,4.046013,4.18732,4.281014


minimum and maximum predicted rating of apps

In [130]:
pred.min(),pred.max()

(2.5122206, 5.100851)

In [131]:
pred_min_index=pred.tolist().index(pred.min())
test_x[pred_min_index:(pred_min_index+1)]

,Category,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated DOM,Last Updated Year,Size_type,Category-Tree,Reviews-Tree,Installs-Tree,Price-Tree,Genres-Tree
8973,6,0,1000,0,0.0,1,36,2764,32,8973,23,2018,0,4.149805,4.215119,4.046013,4.18732,4.163709


In [132]:
pred_max_index=pred.tolist().index(pred.max())
test_x[pred_max_index:(pred_max_index+1)]

,Category,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated DOM,Last Updated Year,Size_type,Category-Tree,Reviews-Tree,Installs-Tree,Price-Tree,Genres-Tree
8831,11,1,10,0,0.0,1,39,1017,18,8831,5,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709


as expected, we can see all apps predicted to have rating just over 5 actually has very few installs

In [133]:
greater5index=[]
for i in [t for t in pred if t >5]:
 greater5index.append(pred.tolist().index(i))

In [134]:
test_x.iloc[greater5index,]

,Category,Reviews,Installs,Type,Price,Content Rating,Genres,Current Ver,Android Ver,app_no,Last Updated DOM,Last Updated Year,Size_type,Category-Tree,Reviews-Tree,Installs-Tree,Price-Tree,Genres-Tree
7440,11,0,10,0,0.0,1,39,113,20,7440,2,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709
7994,11,0,10,0,0.0,1,91,1081,15,7994,6,2018,1,4.208921,4.215119,4.664912,4.18732,4.247114
8831,11,1,10,0,0.0,1,39,1017,18,8831,5,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709
8872,15,0,10,0,0.0,1,62,225,25,8872,7,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709
8988,21,1,10,0,0.0,1,75,113,17,8988,2,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709
9015,15,0,10,0,0.0,1,62,1993,17,9015,9,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709
9209,15,0,10,0,0.0,1,62,1993,17,9209,9,2018,1,4.208921,4.215119,4.664912,4.18732,4.163709
9262,16,0,1,0,0.0,1,65,945,15,9262,25,2018,1,4.208921,4.215119,5.000000,4.18732,4.163709


saving model to pickle

In [136]:
pickle.dump(xgbmmodel,open('model.pkl','wb'))

In [2]:
pd.__version__,np.__version__,sklearn.__version__,sns.__version__

('1.3.5', '1.20.3', '1.0.2', '0.11.2')

In [3]:
sm.__version__,re.__version__,graphviz.__version__

('0.11.1', '2.2.1', '0.16')

In [4]:
import feature_engine

In [5]:
feature_engine.__version__

'1.0.2'